# Pós-Graduação em Ciência de Dados e Analytics - PUC-Rio

## MVP - Engenharia de Dados

### Aluno: Fabiano Ramos (Setembro / 2023)



## Definição do Problema

### Motivação

Para este trabalho, utilizamos o conjunto de dados [Airline Dataset](https://www.kaggle.com/datasets/iamsouravbanerjee/airline-dataset), disponibilizado a partir do [Kaggle](https://www.kaggle.com).

O conjunto de dados é *artificial*, e oferece informações relacionados a operações aéreas, possibilitando a obtenção de *insights* interessantes sobre a demografia e comportamento dos passageiros. Possui um total de 98619 registros e 15 atributos.

Nossas perguntas-alvo foram as seguintes:

* Quais os aeroportos mais movimentados?
* Quais aeroportos possuem um maior número de atrasos e/ou cancelamentos?
* Qual a demografia dos passageiros?

O dicionário de dados é o seguinte:

*   **Passenger ID** - Identificação única do passageiro.
*   **First Name** - Primeiro nome do passageiro.
*   **Last Name** - Último nome do passageiro.
*   **Gender** - Sexo do passageiro. 
*   **Age** - Idade do passageiro.
*   **Nationality** - Nacionalidade do passageiro.
*   **Airport Name** - Nome do aeroporto de partida.
*   **Airport Country Code** - Código do país do aeroporto.
*   **Country Name** - Nome do país do aeroporto.
*   **Airport Continent** - Continente onde o aeroporto está situado.
*   **Continents** - Continente envolvido na rota do aeroporto.
*   **Departure Date** - Data de decolagem.
*   **Arrival Airport** - Aeroporto de destino.
*   **Pilot Name** - Nome do piloto.
*   **Flight Status** - Situação do vôo (cancelado, atrasado, no horário).



### *Stack* de Tecnologia

Decidimos levar a cabo nosso trabalho utilizando a [Amazon Web Services (AWS)](https://aws.amazon.com/).

Como ferramenta de integração de dados, utilizamos o [AWS Glue](https://aws.amazon.com/glue/), um *framework* sem servidor destinado a busca, preparação e integração de dados. Utilizamos também o componente [AWS Glue Data Catalog](https://aws.amazon.com/solutions/analytics/data-catalog/) para preparação do Catálogo de Dados correspondente, o [AWS Athena](https://console.aws.amazon.com/athena) para consultas diretas ao catálogo de dados (com dados ainda armazenados no S3) e também o [Amazon RedShift Serverless](https://aws.amazon.com/redshift/), um serviço de *data warehouse*, também sem servidor, a partir do qual realizamos as análises/consultas desejadas.

A figura abaixo mostra em linhas gerais nosso *pipeline* de dados:

![pipeline](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/pipeline.png)

## Coleta dos Dados

### Análise preliminar

Em primeiro lugar, realizamos uma inspeção do *dataset* para verificar sua qualidade e para apurar se operações de transformação se fariam necessárias.

Fizemos uma [cópia](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/data/Airline_Dataset_Updated.csv) do arquivo *csv* original em nosso github para que este trabalho se mantenha auto-contido e íntegro em caso de consulta futura.


In [ ]:
# carga das bibliotecas
import pandas as pd

# desabilita warnings
import warnings
warnings.filterwarnings('ignore')

# carrega o dataset
dataset = pd.read_csv('https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/data/Airline_Dataset_Updated.csv')

# dimensoes do dataset
print(dataset.shape)

# nulos?
print(dataset.isnull().sum())

# atributos categóricos estão corretos?
print(dataset['Gender'].unique())
print(dataset['Flight Status'].unique())

# ID é mesmo único?
assert(dataset['Passenger ID'].nunique() == dataset.shape[0])

(98619, 15)
Passenger ID            0
First Name              0
Last Name               0
Gender                  0
Age                     0
Nationality             0
Airport Name            0
Airport Country Code    0
Country Name            0
Airport Continent       0
Continents              0
Departure Date          0
Arrival Airport         0
Pilot Name              0
Flight Status           0
dtype: int64
['Female' 'Male']
['On Time' 'Delayed' 'Cancelled']


Como podemos observar, o conjunto de dados é bem comportado (inclusive os atributos categóricos *Flight Status* e *Gender*) e pode ser carregado diretamente sem necessidade de pré-processamento.

### Carga dos Dados para a Nuvem

Para a coleta dos dos dados, criamos um script ETL python, a partir do AWS Glue.

O script é responsável por buscar os [dados](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/data/Airline_Dataset_Updated.csv) diretamente do nosso github e salvá-los dentro de um bucket S3 previamente criado.

O passo a passo da implantação do script foi o seguinte:

1.   Criação de um IAM Role para execução. Criamos um Role chamado *AWSGlue*, composto pelas políticas pré-definidas *AWSGlueServiceRole*, *AWSGlueSchemaRegistryFullAccess* e *AWSGlueConsoleFullAccess*.

![glue-roles](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/glue-roles.png)

2.   Criação do S3 bucket destino. Atribuímos o nome *aws-glue-rawdata*, uma vez que as permissões do *AWSGlueServiceRole* exigem que o nome do bucket possua *aws-glue-* como prefixo.

3.   Criação do script python propriamente dito, que chamamos de *data-collect*, conforme mostramos a seguir. O [script](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/scripts/data-collect.py) também pode ser consultado em nosso github.

![data-collect](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/data-collect.png)

Através da execução do script, os dados de entrada foram então coletados e armazenados no S3 bucket indicado.

![aws-glue-rawdata](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/aws-glue-rawdata.png)

Finalmente, nas configurações de rede, criamos um *endpoint* para que o S3 pudesse ser acessado pelos jobs ETL Glue mais a frente, durante a carga dos dados para o *data warehouse*.

![s3-endpoint](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/s3-endpoint.png)


## Modelagem


Decidimos então manter a representação dos dados em sua forma *flat*, como uma única tabela, como em um Data Lake.

Prosseguimos então para a definição do Catálogo de Dados associado, usando o [AWS Glue Data Catalog](https://aws.amazon.com/solutions/analytics/data-catalog/). Seguimos então os passos descritos a seguir:

1. Criação da base de dados *airline-db*.

![airline-db](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/airline-db.png)


2. Criação da *tabela* , a partir de um *crawler*, com as seguintes configurações:
   - Nome do *crawler*: *airline-tbl-crawler*
   - Fonte de dados: *s3://aws-glue-rawdata/airline-tbl/*
   - IAM role: *AWSGlue*
   - Base de dados: *airline-db*

![crawler](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/crawler.png)

Ao executar o *crawler* a tabela foi criada e seu esquema automaticamente detectado, conforme figura abaixo. O nome da tabela foi automaticamente gerado  a partir do nome da pasta S3 onde o csv estava armazenado, *airline_tbl*  (note que o '-' foi substituído por '_').

![datacatalog-table](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/datacatalog-table.png)

![datacatalog-schema](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/datacatalog-schema.png)

Todos os campos definidos no csv foram corretamente detectados. A seguir, através da opção *Edit schema as JSON*, editamos e ajustamos manualmente o esquema da tabela (nomes de colunas, tipos de dados e comentários), conforme mostramos abaixo. O arquivo [JSON](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/scripts/aws-glue-rawdata.json) completo pode ser encontrado em nosso github. O ajuste do nome das colunas (remoção de espaços) foi fundamental para que a carga para o *data warehouse* (realizada nos passos seguintes) funcionasse corretamente nos passos seguintes.

![datacatalog-schema-edited](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/datacatalog-schema-edited.png)


Um vez disponibilizado o catálogo de dados, pode-se efetuar consultas diretamente sobre os dados persistidos no S3 utilizando o componente [AWS Athena](https://aws.amazon.com/athena/). Neste momento realizamos então apenas uma consulta básica com o objetivo de verificar que o catálogo de dados foi criado corretamente.

Para a correta utilização do Athena, um bucket s3 precisou ser configurado para armazenar os resultados das consultas. Criamos então um bucket chamado *aws-glue-athena-workspace* para tal propósito.

A seguir, realizamos uma consulta simples, para verificar a integridade do catálogo de dados:

![athena-query](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/athena-query.png)




## Carga dos Dados no Data Warehouse




O passo seguinte foi então carregar os dados no [AWS Redshift Serverless](https://us-east-1.console.aws.amazon.com/redshiftv2/home?region=us-east-1#landing).

Realizamos então a configuração inicial do serviço. Criamos um IAM Role apropriado contendo a política *AmazonRedshiftAllCommandsFullAccess*.

![redshift-creation-1](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/redshift-creation-1.png)

![redshift-creation-2](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/redshift-creation-2.png)

Neste momento, voltamos ao Glue Studio para a criação de um job ETL tendo como fonte de dados o Catalógo de Dados previamente criado e, como destino, o Redshift.  Como pré-requisito, foi necessário configurar uma conexão com o RedShift, conforme mostrado abaixo.

![redshift-connection](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/redshift-connection.png)

Uma vez criada a conexão, o job ETL pôde então ser criado, conforme mostrado abaixo. Criamos o job visualmente, mas o [fonte completo](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/scripts/load-redshift.py) do script pode ser observado em nosso github.  Não aplicamos nenhuma transformação de tipos de dados no esquema, mas descartamos os campos *First Name*, *Last Name* e *Pilot Name* que são irrelevantes ao problema.

![redshift-load](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/redshift-load.png)

Após a carga, realizamos então uma consulta de testes a partir do console do RedShift, no sentido de validar a carga:

![redshift-query](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/redshift-query.png)



## Análise dos Dados



### Qualidade dos Dados

Já havíamos realizado a inspeção do *dataset* durante a fase preliminar e não encontramos problemas estruturais.

Notamos, no entanto, que as informações sobre os aeroportos de chegada (código) não são compatíveis (em nível de detalhe) com as dos aeroportos de partida (nome/país), o que limita bastante as oportunidades de análise.

### Solução das Questões Propostas


#### Quais os aeroportos mais movimentados?

A primeira pergunta que tentamos responder foi determinar os aeroportos mais movimentados.  

Realizamos então uma consulta relativamente simples, com o totalizador e deixando de fora os vôos cancelados. Limitamos os resultados a apenas os 10 aeroportos mais movimentados.

A consulta e sua resposta são mostrados na figura abaixo.

In [ ]:
SELECT
    airport_name, airport_country_code, COUNT(*) AS total
FROM
    airline
WHERE
    flight_status != 'Cancelled'
GROUP BY
    airport_name, airport_country_code
ORDER BY
    total DESC
LIMIT 10;




![query-movimentados](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/query-movimentados.png)


#### Quais aeroportos possuem um maior número de atrasos e/ou cancelamentos?


A segunda pergunta que tentamos responder foi determinar os aeroportos mais afetados por atrasos e/ou cancelamentos.

Um fator complicador para a consulta foi que tentamos arbitrariamente filtrar apenas os aeroportos com 20 ou mais vôos, já que este tipo de consulta estatística faz pouco sentido para aeroportos muito pequenos. Este pequeno detalhe fez com que precisássemos realizar uma subconsulta, uma vez que a cláusula WHERE não poderia ser inserida na primeira *query*.

Mais uma vez, limitamos os resultados a apenas os 10 aeroportos mais afetados. 

A consulta e sua resposta são mostrados na figura abaixo.

In [ ]:
WITH flight_problems AS (
    SELECT
        airport_name, airport_country_code,
        SUM(CASE WHEN flight_status = 'Cancelled' OR flight_status = 'Delayed' THEN 1 ELSE 0 END) AS problems,
        COUNT(*) AS total, problems*100.0/total AS percentual
    FROM
        airline
    GROUP BY
        airport_name, airport_country_code
)
SELECT
    airport_name, airport_country_code, problems, total, percentual
FROM
    flight_problems
WHERE
    total > 20
ORDER BY
    percentual DESC
LIMIT 10;

![query-percentual](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/query-percentual.png)


#### Qual a demografia dos passageiros?


Para a terceira pergunta, tentamos entender a  demografia dos passageiros.

Idosos e crianças trazem requisitos diferenciados às empresas aéreas (preço e acomodações) então consideramos esta uma consulta relevante. Incluímos também o sexo dos passageiros, e decidimos agrupar por continente para uma melhor representatividade.

A consulta e sua resposta são mostrados na figura abaixo.

In [ ]:
SELECT
    airport_continent,
    SUM(CASE WHEN Gender = 'Male' THEN 1 ELSE 0 END) AS males,
    SUM(CASE WHEN Gender = 'Female' THEN 1 ELSE 0 END) AS females,
    SUM(CASE WHEN Age > 65 THEN 1 ELSE 0 END) AS seniors,
    SUM(CASE WHEN Age < 12 THEN 1 ELSE 0 END) AS children,
    COUNT(*) AS total
FROM
    airline
GROUP BY
    airport_continent
ORDER BY
    airport_continent;

![query-distribuicao](https://raw.githubusercontent.com/pragmaerror/puc-ciencia-de-dados-e-analytics/main/engenharia-de-dados/mvp/images/query-distribuicao.png)


## Conclusão & Auto Avaliação


As atividades aqui descritas correspondem ao nosso primeiro contato com a atividade de ETL (e Engenharia de Dados de uma forma mais ampla).

Nosso foco pessoal foi o aprendizado da platforma AWS, e desta forma tentamos manter as demais variáveis (*dataset* e escopo do trabalho) o mais controladas possível dado o curto prazo de confecção desde projeto.  Já tínhamos alguma familiaridade com o GCP (embora em outro contexto) e nos desafiamos a aprender a AWS, o que foi bastante recompensador. 

Neste trabalho, fomos capazes de produzir o *pipeline* desejado, desde a análise preliminar dos dados, passando pela sua (simples) modelagem, produção do catálogo de dados correspondente até a sua carga no *data warehouse*, a partir de onde realizamos as consultas desejadas.

Como possibilidades de continuação deste trabalho, podemos destacar em primeiro lugar melhorar a reprodutibilidade da construção do *pipeline*,  uma vez que realizamos muitos passos manuais. Entendemos que técnicas de IaC (*infrastructure as code*) seriam indispensáveis em um ambiente de produção. Mesmo dentro do AWS Glue, identificamos a funcionalidade de *workflows*, que supostamente permitem a definição do fluxo de trabalho através de *blueprints*. Seria bem interessante explorar este caminho no futuro.

Outro desenvolvimento que gostaríamos de realizar no futuro é a interação programática (via *python*) com o AWS Glue e com o Redshift.

Além disso, o uso de dados reais como por exemplo os [dados abertos](https://www.anac.gov.br/acesso-a-informacao/dados-abertos/) da ANAC possibilitariam mais oportunidades de modelagem, transformação e análise. Chegamos examinar estes dados, mas eles apresentaram demasiados problemas e inexatidões de forma que preferimos evitá-los neste momento, por razões já apresentadas.

Finalmente, de forma geral nossa avaliação desta atividade foi bastante positiva. O escopo bastante amplo, ao mesmo tempo que foi um ponto de complexidade adicional, permitiu aos estudantes direcionar suas implementações de acordo com seus interesses e competências individuais.

